In [7]:
import numpy as np
import math

"""Every single word that is input to our transformer is going to have three vectors:
    Query vector: What am I looking for? [sequence length x d_k]
    Key vector: What I can offer? [sequence length x d_k]
    Value vector: What I actually offer? [sequence length x d_v]"""

L, d_k, d_v = 4,8,8 # L-> length of sequence
q = np.random.randn(L,d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)

In [8]:
print("Q:\n",q)
print("K:\n",k)
print("V:\n",v)

Q:
 [[-1.16706321  0.08584256  0.41167449 -1.31804891 -0.63838795 -1.16439709
   0.33402269  0.01780395]
 [-0.76829832  0.36800784 -0.54593039 -0.89403059  1.3478503   0.59814107
   0.43870942  1.5918994 ]
 [-1.37601867  0.6992938   0.93817583 -0.25495501 -0.68275896  0.38011513
  -1.44079193  0.95784556]
 [ 0.6006827  -0.61290058  0.84649809 -0.21517846 -0.62438373  2.45871444
  -0.72788501  0.48967165]]
K:
 [[ 0.10283579 -1.67996589 -0.50757404  0.84824493  0.13783929  0.28868051
  -1.06438927  1.63075758]
 [ 1.38688596 -1.06611274  1.01880174 -0.26574013  0.86551963 -0.56443222
   0.93224403  0.77630729]
 [-0.42241474 -0.17184669 -1.81517834  0.36890399  0.73425581 -0.87597131
  -0.09894674 -1.76780258]
 [-1.03406972  0.19363684  1.69002306 -0.31636521 -2.80383829  0.51330065
   1.14647019  0.19413396]]
V:
 [[ 0.73386825  0.71691135  0.55899857 -0.04507007 -1.04498757 -1.71936946
  -0.06618644 -1.83629274]
 [-1.67348     1.02126157  0.11838808  0.16386213  2.15629617  1.29116667
   

### Self-Attention
To create an initial attention matrix we want every word to look at every other word just to see if it has any affinity to that word or not.

![self-attention-formula.png](self-attention-formula.png)

In [9]:
np.matmul(q,k.T)

array([[-2.34184195, -0.51053031, -0.26854913,  3.91482543],
       [ 1.30899681,  0.69727312, -1.46941106, -2.43417993],
       [ 1.10244651, -3.03541774, -3.72094099,  3.86809038],
       [ 2.67624635,  0.17946999, -5.17017911,  3.03214249]])

In [10]:
# why we need sqrt(d_k) in denominator?
q.var(), k.var(), np.matmul(q,k.T).var()

(0.8597066729257826, 1.1234542303924746, 7.092999622286958)

As we can see the variance of query and key vectors are close to 1 but the variance matmul(q,k.T) is quite high.
That's why we divide it with the square root of the dimension of k and q to stabilize the values and reduce the variance

In [11]:
scaled = np.matmul(q,k.T)/math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(0.8597066729257826, 1.1234542303924746, 0.8866249527858696)

In [12]:
scaled

array([[-0.82796616, -0.18049972, -0.09494646,  1.3840998 ],
       [ 0.46280026,  0.24652328, -0.51951526, -0.86061257],
       [ 0.3897737 , -1.07318224, -1.3155513 ,  1.36757647],
       [ 0.94619597,  0.06345222, -1.82793435,  1.07202426]])

### Masking
* This is to ensure words don't get context from words generated in the future
* Not required in the encoders, but required in the decoders

In [14]:
mask = np.tril(np.ones((L,L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

For example the input is : "My name is Rishav"
* The word 'my' can only look at itself [1,0,0,0]
* The word 'name' can only look at itself and 'my' [1,1,0,0]
* The word 'is' can only look at itself,'my' and 'name' [1,1,1,0]
* The word 'rishav' can only look at itself,'my', 'name', 'is' [1,1,1,1]

In [15]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0

In [16]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [17]:
scaled + mask

array([[-0.82796616,        -inf,        -inf,        -inf],
       [ 0.46280026,  0.24652328,        -inf,        -inf],
       [ 0.3897737 , -1.07318224, -1.3155513 ,        -inf],
       [ 0.94619597,  0.06345222, -1.82793435,  1.07202426]])

### Softmax
Softmax converts a vector into a probability distribution

In [18]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

In [20]:
attention = softmax(scaled + mask)
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.55385947, 0.44614053, 0.        , 0.        ],
       [0.70758182, 0.16384114, 0.12857704, 0.        ],
       [0.38312157, 0.15847686, 0.02390821, 0.43449335]])

In [21]:
new_v = np.matmul(attention, v)
new_v

array([[ 7.33868250e-01,  7.16911353e-01,  5.58998570e-01,
        -4.50700749e-02, -1.04498757e+00, -1.71936946e+00,
        -6.61864358e-02, -1.83629274e+00],
       [-3.40147383e-01,  8.52694323e-01,  3.62424372e-01,
         4.81430507e-02,  3.83234864e-01, -3.76247266e-01,
        -2.74469350e-02, -1.08637426e+00],
       [ 1.24975498e-03,  5.51117445e-01,  6.01514700e-01,
        -2.12698628e-02, -3.17155206e-01, -1.04633677e+00,
         5.08011133e-02, -1.49140075e+00],
       [-1.82752640e-01,  8.97634529e-02,  7.40971645e-01,
        -3.04723488e-01, -4.51413531e-01, -7.10563447e-01,
        -2.96123954e-01, -1.01332457e+00]])

In [22]:
v

array([[ 0.73386825,  0.71691135,  0.55899857, -0.04507007, -1.04498757,
        -1.71936946, -0.06618644, -1.83629274],
       [-1.67348   ,  1.02126157,  0.11838808,  0.16386213,  2.15629617,
         1.29116667,  0.02064607, -0.15539082],
       [-1.89642874, -0.96036319,  1.45111941, -0.12620026,  0.53640187,
        -0.32111825,  0.73302944, -1.29583   ],
       [-0.35297505, -0.74520476,  1.08943391, -0.71441209, -0.9335096 ,
        -0.5725717 , -0.6710433 , -0.58503629]])

## Function

In [23]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention

In [24]:
values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)

Q
 [[-1.16706321  0.08584256  0.41167449 -1.31804891 -0.63838795 -1.16439709
   0.33402269  0.01780395]
 [-0.76829832  0.36800784 -0.54593039 -0.89403059  1.3478503   0.59814107
   0.43870942  1.5918994 ]
 [-1.37601867  0.6992938   0.93817583 -0.25495501 -0.68275896  0.38011513
  -1.44079193  0.95784556]
 [ 0.6006827  -0.61290058  0.84649809 -0.21517846 -0.62438373  2.45871444
  -0.72788501  0.48967165]]
K
 [[ 0.10283579 -1.67996589 -0.50757404  0.84824493  0.13783929  0.28868051
  -1.06438927  1.63075758]
 [ 1.38688596 -1.06611274  1.01880174 -0.26574013  0.86551963 -0.56443222
   0.93224403  0.77630729]
 [-0.42241474 -0.17184669 -1.81517834  0.36890399  0.73425581 -0.87597131
  -0.09894674 -1.76780258]
 [-1.03406972  0.19363684  1.69002306 -0.31636521 -2.80383829  0.51330065
   1.14647019  0.19413396]]
V
 [[ 0.73386825  0.71691135  0.55899857 -0.04507007 -1.04498757 -1.71936946
  -0.06618644 -1.83629274]
 [-1.67348     1.02126157  0.11838808  0.16386213  2.15629617  1.29116667
   0.0